Setup

In [1]:
!pip install transformers datasets peft torch sacrebleu --quiet

In [2]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, load_metric
import numpy as np
import torch, os

2024-08-01 13:34:25.787596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 13:34:25.787704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 13:34:25.933129: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"

In [4]:
raw_dataset = load_dataset('ai4bharat/samanantar', 'hi', split='train[:10000]', trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = raw_dataset.take(10000)

In [6]:
dataset

Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 10000
})

### Prepare the dataset for the model

In [7]:
tokenized_datasets = dataset.train_test_split(test_size=0.1)
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']

In [8]:
train_dataset[5]
eval_dataset[5]

{'idx': 3419,
 'src': 'Bigg Boss 14 contestant Rubina Dilaik is the winner of the season.',
 'tgt': 'बिग बॉस 14 सीजन के विनर का खिताब रुबीना दिलैक ने जीत लिया है.'}

### Load the Model and Tokenizer

In [9]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [10]:
#Define a function to tokenize the dataset
def preprocess_function(examples):
    inputs = [ex for ex in examples['src']]
    targets = [ex for ex in examples['tgt']]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]
    
    model_inputs["labels"] = labels
    return model_inputs

In [11]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    logging_dir='./logs',           # Directory for storing logs
    logging_steps=10,               # Log every 10 steps
    generation_max_length=128,      # Set maximum length for generation
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    return result

/tmp/ipykernel_34/1540555718.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


The repository for sacrebleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sacrebleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [15]:
save_directory = "/kaggle/new_model/saved_model"

os.makedirs(save_directory, exist_ok=True)


In [16]:
#Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,2.059300,2.027637,18.694625,"[9233, 4478, 2409, 1341]","[18784, 17784, 16785, 15792]","[49.15353492333901, 25.179937022042285, 14.352100089365505, 8.49164133738602]",0.948610,18784,19775


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation para

TrainOutput(global_step=563, training_loss=2.2042956157433626, metrics={'train_runtime': 1416.5948, 'train_samples_per_second': 6.353, 'train_steps_per_second': 0.397, 'total_flos': 2438020988928000.0, 'train_loss': 2.2042956157433626, 'epoch': 1.0})

### Save the Finetuned Model and Tokenizer

In [24]:
# save the trained model
trainer.save_model('/.finetuned-MBart-en-ta')
# save model and tokenizer 
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


Model saved to /kaggle/new_model/saved_model


### Load the Finetuned model and check out the translation

In [26]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

#Load your model  and tokenizer
model_name = save_directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Example English sentence to translate
english_sentence = "I think this project will go really well"

#Tokenize the input sentence
inputs = tokenizer(english_sentence, return_tensors ="pt").to(device)

#Generate the transaltion
with torch.no_grad():
    outputs = model.generate(**inputs, max_length = 128, num_beams = 4, early_stopping = True)
    
#Decode the generated tokens
translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"English: {english_sentence}")
print(f"Hindi Translation: {translated_sentence}")

English: I think this project will go really well
Hindi Translation: मुझे लगता है कि यह परियोजना अच्छी तरह से चलेगी
